<a href="https://colab.research.google.com/github/ancestor9/Gyeongbok-AI-study/blob/main/2025%20AI%20Tutor/%EC%A0%9C2%EA%B0%95_gpt4o_mini_WebBaseLoader_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part2: RAG (Retrieval-Augmented Generation)
- 모델의 학습 데이터에 포함되지 않은 데이터를 사용 (환각 방지)
- **외부 데이터**를 검색(retrieval)한 후, 생성(generation) 단계에서 LLM에 전달
- 모델은 주어진 컨텍스트나 질문에 더 적합하고 풍부한 정보를 반영한 답변을 생성
- 논문: https://arxiv.org/abs/2005.11401

## 0. 환경 구성

### 1) 라이브러리 설치

In [1]:
!pip install -q langchain langchain-openai langchain_community tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.9 MB/s eta 0:00:00

In [2]:
import langchain

langchain.__version__

'0.3.23'

### 2) OpenAI 인증키 설정
https://openai.com/

In [3]:
from google.colab import userdata
apikey  = userdata.get('openapi')

import os
os.environ['OPENAI_API_KEY'] = apikey

## 1. RAG 파이프라인 개요
- Load Data - Text Split - Indexing - Retrieval - Generation

### **1.1. WebBaseLoader (Loading from URL)**

##### Step 1: Load Data

In [4]:
# Data Loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader

# 위키피디아 정책과 지침
url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
loader = WebBaseLoader(url)

# 웹페이지 텍스트 -> Documents
docs = loader.load()

print(len(docs))
print(len(docs[0].page_content))
print(docs[0].page_content[5000:6000])

1
13300
공동체의 규범을 총체적으로 어기고 있다면 규범 준수를 위해 좀 더 빠르게 강력한 수단을 이용해야 합니다. 특히 정책 문서에 명시된 원칙을 지키지 않는 것은 대부분의 경우 다른 사용자에게 받아들여지지 않습니다 (다른 분들에게 예외 상황임을 설득할 수 있다면 가능하기는 하지만요). 이는 당신을 포함해서 편집자 개개인이 정책과 지침을 직접 집행 및 적용한다는 것을 의미합니다.
특정 사용자가 명백히 정책에 반하는 행동을 하거나 정책과 상충되는 방식으로 지침을 어기는 경우, 특히 의도적이고 지속적으로 그런 행위를 하는 경우 해당 사용자는 관리자의 제재 조치로 일시적, 혹은 영구적으로 편집이 차단될 수 있습니다. 영어판을 비롯한 타 언어판에서는 일반적인 분쟁 해결 절차로 끝낼 수 없는 사안은 중재위원회가 개입하기도 합니다.

문서 내용
정책과 지침의 문서 내용은 처음 읽는 사용자라도 원칙과 규범을 잘 이해할 수 있도록 다음 원칙을 지켜야 합니다.

명확하게 작성하세요. 소수만 알아듣거나 준법률적인 단어, 혹은 지나치게 단순한 표현은 피해야 합니다. 명확하고, 직접적이고, 모호하지 않고, 구체적으로 작성하세요. 지나치게 상투적인 표현이나 일반론은 피하세요. 지침, 도움말 문서 및 기타 정보문 문서에서도 "해야 합니다" 혹은 "하지 말아야 합니다" 같이 직접적인 표현을 굳이 꺼릴 필요는 없습니다.
가능한 간결하게, 너무 단순하지는 않게. 정책이 중언부언하면 오해를 부릅니다. 불필요한 말은 생략하세요. 직접적이고 간결한 설명이 마구잡이식 예시 나열보다 더 이해하기 쉽습니다. 각주나 관련 문서 링크를 이용하여 더 상세히 설명할 수도 있습니다.
규칙을 만든 의도를 강조하세요. 사용자들이 상식대로 행동하리라 기대하세요. 정책의 의도가 명료하다면, 추가 설명은 필요 없죠. 즉 규칙을 '어떻게' 지키는지와 더불어 '왜' 지켜야 하는지 확실하게 밝혀야 합니다.
범위는 분명히, 중복은 피하기. 되도록 앞부분에서 정책 및 지침의 목적과 범위를 분명하게 밝혀야 합니다. 독자 대

##### Step 2: Split texts

In [5]:
# Text Split (Documents -> small chunks: Documents)
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(len(splits))
print(splits[10])

19
page_content='제안과 채택
 백:아님 § 관료주의  문서를 참고하십시오. 단축백:제안
제안 문서란 정책과 지침으로 채택하자고 의견을 묻는 문서이나 아직 위키백과 내에 받아들여지는 원칙으로 확립되지는 않은 문서입니다. {{제안}} 틀을 붙여 공동체 내에서 정책이나 지침으로 채택할 지 의견을 물을 수 있습니다. 제안 문서는 정책과 지침이 아니므로 아무리 실제 있는 정책이나 지침을 요약하거나 인용해서 다른 문서에 쓴다고 해도 함부로 정책이나 지침 틀을 붙여서는 안 됩니다.
'제안'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.
좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.
정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다.' metadata={'source': '

In [6]:
# page_content 속성
splits[10].page_content

'제안과 채택\n\xa0백:아님 §\xa0관료주의  문서를 참고하십시오. 단축백:제안\n제안 문서란 정책과 지침으로 채택하자고 의견을 묻는 문서이나 아직 위키백과 내에 받아들여지는 원칙으로 확립되지는 않은 문서입니다. {{제안}} 틀을 붙여 공동체 내에서 정책이나 지침으로 채택할 지 의견을 물을 수 있습니다. 제안 문서는 정책과 지침이 아니므로 아무리 실제 있는 정책이나 지침을 요약하거나 인용해서 다른 문서에 쓴다고 해도 함부로 정책이나 지침 틀을 붙여서는 안 됩니다.\n\'제안\'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.\n좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.\n정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다.'

In [7]:
# metadata 속성
splits[10].metadata

{'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8',
 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전',
 'language': 'ko'}

##### Step 3: Indexing

In [8]:
# Indexing (Texts -> Embedding -> Store)
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())

docs = vectorstore.similarity_search("격하 과정에 대해서 설명해주세요.")
# 디폴트로 model_name="text-embedding-ada-002"가 사용됨

print(len(docs))
print(docs[0].page_content)

4
격하
특정 정책이나 지침이 편집 관행이나 공동체 규범이 바뀌며 쓸모없어질 수 있고, 다른 문서가 개선되어 내용이 중복될 수 있으며, 불필요한 내용이 증식할 수도 있습니다. 이 경우 편집자들은 정책을 지침으로 격하하거나, 정책 또는 지침을 보충 설명, 정보문, 수필 또는 중단 문서로 격하할 것을 제안할 수 있습니다. 
격하 과정은 채택 과정과 비슷합니다. 일반적으로 토론 문서 내 논의가 시작되고 프로젝트 문서 상단에 {{새로운 토론|문단=진행 중인 토론 문단}} 틀을 붙여 공동체의 참여를 요청합니다. 논의가 충분히 이루어진 후, 제3의 편집자가 토론을 종료하고 평가한 후 상태 변경 총의가 형성되었는지 판단해야 합니다. 폐지된 정책이나 지침은 최상단에 {{중단}} 틀을 붙여 더 이상 사용하지 않는 정책/지침임을 알립니다.
소수의 공동체 인원만 지지하는 수필, 정보문 및 기타 비공식 문서는 일반적으로 주된 작성자의 사용자 이름공간으로 이동합니다. 이러한 논의는 일반적으로 해당 문서의 토론란에서 이루어지며, 간혹 위키백과:의견 요청을 통해 처리되기도 합니다.

같이 보기
위키백과:위키백과의 정책과 지침 목록
위키백과:의견 요청
수필

위키백과:제품, 절차, 정책
위키백과:위키백과 공동체의 기대와 규범
기타 링크


##### Step 4: Retrieval ~ Generation

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# Rretriever
retriever = vectorstore.as_retriever()

# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
rag_chain.invoke("격하 과정에 대해서 설명해주세요.")

'격하 과정은 특정 정책이나 지침이 더 이상 유효하지 않거나 쓸모없게 되었을 때 이를 낮추는 절차입니다. 이 과정은 채택 과정과 유사하게 진행됩니다. 일반적으로 토론 문서 내에서 논의가 시작되며, 프로젝트 문서 상단에 {{새로운 토론|문단=진행 중인 토론 문단}} 틀을 붙여 공동체의 참여를 요청합니다. 충분한 논의가 이루어진 후, 제3의 편집자가 토론을 종료하고 평가하여 상태 변경에 대한 총의가 형성되었는지 판단합니다. \n\n격하된 정책이나 지침은 최상단에 {{중단}} 틀을 붙여 더 이상 사용되지 않음을 알립니다. 소수의 공동체 인원만 지지하는 비공식 문서는 일반적으로 주된 작성자의 사용자 이름공간으로 이동됩니다. 이러한 논의는 해당 문서의 토론란에서 이루어지며, 때로는 위키백과:의견 요청을 통해 처리되기도 합니다.'

### **1.2. RecursiveCharacterTextSplitter (Loading CSV 파일)**

In [10]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the CSV file into a DataFrame
file_path = '/content/식품안전정보DB(2023)_sample.xlsx'  # Replace with your CSV file path
data = pd.read_excel(file_path)
data

FileNotFoundError: [Errno 2] No such file or directory: '/content/식품안전정보DB(2023)_sample.xlsx'

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define a simple Document class
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

# Create documents with page_content and metadata
documents = []
for _, row in data.iterrows():
    document = Document(
        page_content=row['내용'],
        metadata={
            '국가명': row['국가명'],
            '수집일자': row['수집일자'],
            '제목': row['제목'],
            '정보제공국가': row['정보제공국가'],
            '정보구분': row['정보구분'],
            '식품등유형': row['식품등유형'],
            '원인요소': row['원인요소']
        }
    )
    documents.append(document)

# Initialize the text splitter with specified chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the documents into smaller chunks
splits = text_splitter.split_documents(documents)

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini")

# Initialize the embedding model without the model_name argument
embedding_model = OpenAIEmbeddings()

# Convert text documents into embeddings and store them using Chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

# Convert text documents into embeddings and store them using Chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)


Number of documents found: 4
Sample document content:


TypeError: 'Document' object is not subscriptable

In [ ]:

# Perform a similarity search on the stored embeddings
query = "2023년 국가별 위애요인 기사 건수가 제일 많은 상위 국가는?."
results = vectorstore.similarity_search(query)
results


[Document(metadata={'국가명': '베트남', '수집일자': '2023-12-12', '식품등유형': '축산물>식육류>식육류일반', '원인요소': '안전위생>안전>안전', '정보구분': '글로벌 동향정보', '정보제공국가': '베트남', '제목': '베트남 농업농촌개발부, 식품 및 질병안전 보장을 위한 동물 도축 통제 등 관리 강화'}, page_content='베트남 농업농촌개발부는 12월 8일, 식품안전, 질병안전 보장을 위한 동물 도축의 관리 강화에 관한 공문(số 9001/BNN-TY)을 중앙 소속 각 성(省), 시 인민위원회에 발행했음. 2023년 11월말까지 전국 63개의 성/시 중 32개의 성/시에서 집중 도축망을 구축을 위한 계획을 수립했음. 그러나 전국에는 여전히 24,000개 이상의 소규모 도축장이 존재하며, 그 중 약 18.6%만 수의 기관이 통제하고 있어 식품안전이 보장되지 않을 위험이 매우 높음.농업농촌개발부는 수의국에 동물운송검역, 위생검사, 도축통제관리 및 도축시설에서 금지물질 사용 등에 대하여 각 지방까지 업무단(đoàn công tác)을 설립하도록 요청했음. 또한, 축산국에는 각 지방의 가축 및 수의 전문 기관과 협력하여 축산 시설에서 금지물질 사용 점검 및 감독단 설립을 주관하도록 하였음. (*)자세한 내용은 원문참조'),
 Document(metadata={'국가명': '베트남', '수집일자': '2023-12-12', '식품등유형': '축산물>식육류>식육류일반', '원인요소': '안전위생>안전>안전', '정보구분': '글로벌 동향정보', '정보제공국가': '베트남', '제목': '베트남 농업농촌개발부, 식품 및 질병안전 보장을 위한 동물 도축 통제 등 관리 강화'}, page_content='베트남 농업농촌개발부는 12월 8일, 식품안전, 질병안전 보장을 위한 동물 도축의 관리 강화에 관한 공문(số 9001/BNN-TY)을 중앙 소속 각 성(省), 시 인민위원회에 발행했음. 2023년 11월말까지 전국 63

In [ ]:
results

[Document(metadata={'국가명': '베트남', '수집일자': '2023-12-18', '식품등유형': '식품일반>식품일반>식품일반', '원인요소': '안전위생>안전>안전', '정보구분': '글로벌 동향정보', '정보제공국가': '베트남', '제목': '베트남 식품안전국, 2024년 설 명절 기간 식품안전보장 활동 계획 발표'}, page_content='베트남 식품안전국은 2023년 12월 18일, 2024년 설 명절 및 봄 축제 기간 동안 식품안전 보장을 위한 식품안전에 관한 중앙지시위원회는 다음과 같이 활동 계획을 발표했음.Ⅰ. 목표  -2024년 설 명절 및 봄 축제기간 중 식품의 생산, 가공, 거래, 수입 및 사용 시 식품안전보장 통제를 강화하고 가짜 및 불량 식품의 생산 및 거래를 적시에 예방함.-2024년 설명절 및 봄 축제기간 식중독 발생 최소화II. 기간 및 범위 - 기간: 2023년 12월 20일부터 2023년 3월 20일까지- 범위: 전국 (*)자세한 내용은 원문참조'),
 Document(metadata={'국가명': '베트남', '수집일자': '2023-12-18', '식품등유형': '식품일반>식품일반>식품일반', '원인요소': '안전위생>안전>안전', '정보구분': '글로벌 동향정보', '정보제공국가': '베트남', '제목': '베트남 식품안전국, 2024년 설 명절 기간 식품안전보장 활동 계획 발표'}, page_content='베트남 식품안전국은 2023년 12월 18일, 2024년 설 명절 및 봄 축제 기간 동안 식품안전 보장을 위한 식품안전에 관한 중앙지시위원회는 다음과 같이 활동 계획을 발표했음.Ⅰ. 목표  -2024년 설 명절 및 봄 축제기간 중 식품의 생산, 가공, 거래, 수입 및 사용 시 식품안전보장 통제를 강화하고 가짜 및 불량 식품의 생산 및 거래를 적시에 예방함.-2024년 설명절 및 봄 축제기간 식중독 발생 최소화II. 기간 및 범위 - 기간: 2023년 12월 20일부터 2023년 3월 20일까지- 